In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Simplified data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Normalize the images to the range the pre-trained model expects
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

# Create the base model from the pre-trained EfficientNetB0
base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
model = models.Sequential([
    data_augmentation,
    layers.Lambda(preprocess_input),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 30  # Keeping a higher number of epochs with early stopping
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 800 files belonging to 20 classes.
Found 200 files belonging to 20 classes.
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 426ms/step - accuracy: 0.2907 - loss: 2.5268 - val_accuracy: 0.7350 - val_loss: 0.8526 - learning_rate: 0.0010
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 342ms/step - accuracy: 0.8716 - loss: 0.5099 - val_accuracy: 0.8350 - val_loss: 0.5297 - learning_rate: 0.0010
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 354ms/step - accuracy: 0.9195 - loss: 0.3053 - val_accuracy: 0.8800 - val_loss: 0.3702 - learning_rate: 0.0010
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 338ms/step - accuracy: 0.9636 - loss: 0.1563 - val_accuracy: 0.8950 - val_loss: 0.2763 - learning_rate: 0.0010
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 335ms/step - accuracy: 0.9601 - loss: 0.1280 - val_accuracy: 0.9300 - val_loss: 0.2964 - learning_rate: 0.0010
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 335ms/step - accuracy: 0.9810 - loss: 0.0943 - val_accuracy: 0.9350 - val_loss: 0.2520 - learning_rate: 0.0010
Epoch

In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define paths
train_dir = 'dataset/train'
test_dir = 'dataset/test'

# Load datasets
batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

test_dataset = image_dataset_from_directory(
    test_dir,
    shuffle=True,
    batch_size=batch_size,
    image_size=(img_height, img_width)
)

# Retrieve class names
class_names = train_dataset.class_names

# Prefetch the data for optimal performance
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

base_model = tf.keras.applications.Xception(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights='imagenet'
)
preprocess_input = tf.keras.applications.xception.preprocess_input

# Unfreeze some layers of the base model for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Define the number of classes
num_classes = len(class_names)

# Create the model
inputs = layers.Input(shape=(img_height, img_width, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

# # Create the model
# model = models.Sequential([
#     data_augmentation,
#     layers.Lambda(preprocess_input),
#     base_model,
#     layers.GlobalAveragePooling2D(),
#     layers.Dense(512, activation='relu'),
#     layers.Dropout(0.5),
#     layers.Dense(num_classes, activation='softmax')
# ])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Define callbacks for early stopping and learning rate reduction
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model
epochs = 30  # Increased number of epochs with early stopping
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')

Found 800 files belonging to 20 classes.
Found 200 files belonging to 20 classes.
Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.0846 - loss: 3.7230 - val_accuracy: 0.3000 - val_loss: 2.6904 - learning_rate: 1.0000e-04
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.3831 - loss: 2.1865 - val_accuracy: 0.4650 - val_loss: 2.3166 - learning_rate: 1.0000e-04
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.6174 - loss: 1.3108 - val_accuracy: 0.6450 - val_loss: 1.9861 - learning_rate: 1.0000e-04
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.7028 - loss: 0.9407 - val_accuracy: 0.7050 - val_loss: 1.6858 - learning_rate: 1.0000e-04
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.8070 - loss: 0.6563 - val_accuracy: 0.7550 - val_loss: 1.4035 - learning_rate: 1.0000e-04
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.8489 - loss: 0.4902 - val_accuracy: 0.8000 - val_loss: 1.1803 - learning_rate: 1.000

In [23]:
# # Save the model using the SavedModel format
# model.save('indonesian_food_classifier_saved_model.keras')

# # Load the saved model
# model = tf.keras.models.load_model('indonesian_food_classifier_saved_model.keras')

# # Convert the model to TFLite format with optimizations
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# # Save the optimized TFLite model
# with open('indonesian_food_classifier_optimized.tflite', 'wb') as f:
#     f.write(tflite_model)